# Control any Robotic Arm with ROS
 If you wanted to develop a robot, you had to build a complete system: a physical device, of course, but also the control systems, interface, and inspection tools required to get the robot up and running as a test platform.

We have to:
- Create the Robot arm model
- Import the Robot arm model in ROS
- Control the kinematics 

### Install some packages

First of all we need to install:
- urdf tutorial package
- the moveit package.

The simplest way to install MoveIt is from pre-built binaries (Debian):

In [ ]:
rosdep update
sudo apt-get update
sudo apt-get install python3
sudo apt-get install ros-melodic-urdf-tutorial
sudo apt install ros-melodic-moveit

#### Documentation
Interesting links are:
- ROS Tutorial: Create an arm on a mobile robot using Moveit!: https://www.youtube.com/watch?v=l4dtSRvlAjg
- Control any Robotic Arm with ROS: https://blog.usejournal.com/control-any-robotic-arm-with-ros-b10a3115306c
- Udemy course: https://www.udemy.com/course/robotics-with-ros-build-robotic-arm-in-gazebo-and-moveit/
- Udemy course repository: https://github.com/noshluk2/4_Robotic-Arm-Manupilation-using-Gazbo-and-Moveit
- How to control a robot arm with the Arduino and ROS: https://maker.pro/arduino/tutorial/how-to-control-a-robot-arm-with-ros-and-arduino
- Moveit tutorials: http://docs.ros.org/en/melodic/api/moveit_tutorials/html/index.html
- Moveit Setup Assistant Tutorial: http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

Public repositories in:
- https://bitbucket.org/theconstructcore/
- https://bitbucket.org/theconstructcore/my-robotic-manipulator/src/master/
- https://bitbucket.org/theconstructcore/two-wheeled-robot-motion-planning/src/master/


## Create the Robot arm model

First of all, we create a "arm_kinematics_ws" where we will install the packages:
- robot_description package: where the URDF model of robotic arm is located. This URDF file is created from scratch or obtained from a SolidWorks plugin
- robot kinematics control package: the moveit_config package

Create a new repository "arm_kinematics_ws" in your github

Syncronise the repository in your local computer

Create the src folder and compile the workspace

### Design a 3D robot arm model from scratch

ROS and MoveIt require our robotic arm model files to be in URDF. The Unified Robotic Description Format (URDF) is an XML file format used in ROS to describe all elements of a robot. It lists all the kinematic properties of the links and joints in a chain from base to tip. ROS also accepts .xacro files which are very similar to urdf and are easier to write.

We can make use of any robot arm design files available on the internet in CAD neutral format (such as STL). Alternatively, one can design it from scratch using any of the popular CAD software packages such as SolidWorks.

First of all you need to create a new package for robot description

In [ ]:
catkin_create_pkg custom_robotic_arm rospy std_msgs

Create "urdf" and "launch" folders inside the new created package

### Generate URDF model file from SolidWorks
Fortunately, due to a few talented ROS developers, there exists a convenient plugin in Solidworks to export our assembly in URDF format.

Assuming you have your model completely set up as a SolidWorks assembly, we now use the SolidWorks to URDF Exporter: http://wiki.ros.org/sw_urdf_exporter

Follow the tutorial to obtain the URDF file: https://blog.usejournal.com/control-any-robotic-arm-with-ros-b10a3115306c

The built package will contain directories for meshes, textures and robots. It will also contain a ROS package.xml (manifest) file so you can use this as a ROS package by just copying it to your ROS system. The path locations in the URDF are relative to the package itself. Use proper ROS package names to avoid errors.

## Import the Robot arm model in ROS

## Install robot_description package

For this tutorial, make sure you have the Franka description package for Melodic:

In [ ]:
sudo apt-get install ros-melodic-franka-description

Copy the folder "franka_description" to src folder

Compile again the workspace

Verify the lasts lines in .bashrc file

In [ ]:
source /opt/ros/melodic/setup.bash
source /media/sf_github_manelpuig/arm_kinematics_ws/devel/setup.bash

## Create a moveit_config package

To start the MoveIt Setup Assistant:

In [ ]:
roslaunch moveit_setup_assistant setup_assistant.launch

Follow the "Moveit Setup Assistant" Tutorial to create the moveit_config package for the panda robot arm

http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

You have now the panda_moveit_config package properly installed

Carefully:
- add the links and joints "Planning groups"
- add the urdf file generated in the robots folder
You need some modifications to this package:
- add the created urdf file in robots folder
- change the gazebo.launch file

## Robot arm Kinematics control 
The quickest way to get started using MoveIt is through its RViz plugin. Rviz is the primary visualizer in ROS and an incredibly useful tool for debugging robotics.

http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/quickstart_in_rviz/quickstart_in_rviz_tutorial.html

In [ ]:
roslaunch panda_moveit_config demo.launch rviz_tutorial:=true

Some important configuration parameters:
- In Context Motion Planning Menu: Planning Library OMPL select RTTConnect
- In Planning Request: Interactive marker size to 0,3
- Select "Allow External Communication to use a joystick

<img src="./Images/1_panda_kine_rviz1.png">

In [ ]:
roslaunch gazebo_ros empty_world.launch paused:=true use_sim_time:=false gui:=true throttled:=false recording:=false debug:=true

In [ ]:
rosrun gazebo_ros spawn_model -file /media/sf_github_manelpuig/arm_kinematics_ws/src/franka_description/robots/panda_arm.urdf -urdf -x 0 -y 0 -z 0 -model panda

Change the gazebo.launch file considering:
- urdf path to the urdf file and not the xacro file
- the model has to be changed to panda (not robot as defauld)